In [261]:
import numpy as np

In [262]:
# Import ordering table and the update list
ordering_and_updates = open('ordering_list_and_updates.txt', 'r').read()
ordering_and_updates_splited = ordering_and_updates.split('\n\n')

ordering_list = ordering_and_updates_splited[0]
updates_list = ordering_and_updates_splited[1]

ordering_list = ordering_list.split('\n')
updates_list = updates_list.split('\n')[:-1]
for i, ordering in enumerate(ordering_list) :
    ordering_list[i] = ordering.split('|')
for i, pages in enumerate(updates_list):
    updates_list[i] = pages.split(',')


for i in range(len(ordering_list)):
    for j, page in enumerate(ordering_list[i]):
        ordering_list[i][j] = int(page)

for i in range(len(updates_list)):
    for j, page in enumerate(updates_list[i]):
        updates_list[i][j] = int(page)

ordering = np.array(ordering_list)

# print(ordering, '\n', updates_list)

In [263]:
### Part 1 - Compute the sum of the middle pages over all correctly ordered tables

# Dictionnary where the keys are pages with ordering mentions 
# and values are the pages that must be prior to this page in the updating process.

# Initialize the dictionnary
order_dict = {}

# For each rule in the ordering table
for order in ordering :
    prior = order[0]
    posterior = order[1]
    # If the posterior page in the rule hasn't been seen yet
    if posterior not in order_dict.keys():
        order_dict[posterior] = []
        order_dict[posterior].append(prior) # Add the prior page to the dictionnary
    
    # If the prior page of the rule hasn't been seen for the current posterior page
    if prior not in order_dict[posterior]:
        order_dict[posterior].append(prior) # Add the prior page to the dictionnary
    

In [264]:
# Initialize the True/False table of the updates
updates_check_list = []

# Iterate over all the updates
for update in updates_list:
    updates_check_list.append(True)     # First suppose that the update is correct
    for i, page in enumerate(update):   # For all the pages in the update
        for posterior_page in update[i+1:]:     # Iterate over all the posterior pages
            if posterior_page in order_dict[page]:  # If one of the posterior pages should be prior to the current page wrt the ordering table
                updates_check_list[-1] = False  # Turn the check to False
                break 

        
        if updates_check_list[-1] == False:
            break



In [270]:
# Count the sum of the middle pages over all correctly ordered updates
true_updates_indices = np.where(updates_check_list)[0]
sum = 0


for index in true_updates_indices:
    mid_page_index = len(updates_list[index])//2 # Find the middle page index 
    mid_page_value = updates_list[index][mid_page_index]    # Find the middle page value 
    sum += mid_page_value

print("Sum of the middle pages over all the ordered updates :",sum)


Sum of the middle pages over all the ordered updates : 4569


In [266]:
## Re-order the pages of the uncorrect updates

# Extract the indices of the uncorrect updates
false_updates_indices = np.where(np.array(updates_check_list) == False)[0]
false_updates_list = [updates_list[index] for index in false_updates_indices]

for n in range(3):

    # For each updates
    for update in false_updates_list:

        # For each pages
        for i, page in enumerate(update):

            # Iterate over each posterior pages wrt the current page
            for j, posterior_page in enumerate(update[i+1:]):

                # If the posterior page should be before the current page
                if posterior_page in order_dict[page]:
                    update.pop(i+1+j)   # Pop the posterior page of the update
                    edit = False    # Set an editing marker 

                    # Iterate through the pages prior to the current page
                    for k, prior_page in enumerate(update[:i+1]):

                        # If the posterior page should be before the prior page wrt to the ordering table
                        if posterior_page in order_dict[prior_page]:
                            update.insert(k,posterior_page) # Insert the posterior page before the prior page
                            edit = True # Turn the marker to True

                        # If the posterior page has been re-inserted
                        if edit :
                            break   # Break the loop

                    # If the posterior page hasn't been inserted before any prior pages
                    if not edit :
                        update.insert(i, posterior_page)    # Insert it before the current page

In [268]:
# Count the sum of the middle pages over all the re-ordered updates

sum = 0

for update in false_updates_list:
    mid_page_index = len(update)//2     # Find the middle page index 
    mid_page_value = update[mid_page_index]     # Find the middle page value 
    sum += mid_page_value

print("Sum of the middle pages over all the re-ordered updates :",sum)

Sum of the middle pages over all the re-ordered updates : 6456
